In [1]:
import sys
import os
print(sys.prefix)
print(os.environ['CONDA_DEFAULT_ENV'])

C:\Users\andrew.domenico\AppData\Local\Continuum\anaconda3
base


In [2]:
from __future__ import print_function
#from ortools.constraint_solver import routing_enums_pb2
#from ortools.constraint_solver import pywrapcp
import pandas as pd
import numpy as np
import os
from IPython.display import display
from math import sin, cos, sqrt, atan2, radians
from geopy import distance
from geopy.distance import great_circle
from geopy.distance import vincenty
from sklearn.neighbors import DistanceMetric


taskFile = "C:/Users/andrew.domenico/OneDrive - Accenture/Documents/CTL/Dispatch Optimization/task.csv"

#import the task file
pwd = os.getcwd()
os.chdir(os.path.dirname(taskFile))
taskData = pd.read_csv(os.path.basename(taskFile))
os.chdir(pwd)
pd.options.display.max_columns = None

In [3]:
#drop columns with 100% missing data
taskData = taskData[['SerialNo','ExternalRefID','CallID','CTLDispatchCallID','CTLAggregationID',
                     'Number','EarlyStart','DueDate','Priority','Area','Region','District','Postcode','TaskType', 
                     'Duration','NumberOfRequiredEngineers','AppointmentStart','AppointmentFinish',
                     'Latitude','Longitude','Street','City','State','CountryID','CTLDispatchReady', 
                     'ES+duration','AS+duration','(AS+duration>=DD','(ES+duration>=DD','Duration (hh:mm:ss)', 
                     'DD-ES','ES>=DD','DD-AS','AS>=DD','AF>DD','Note']]



In [4]:
#split date and time elements into separate columns
earlyStart = taskData["EarlyStart"].str.split("T", n = 1, expand = True)
taskData["EarlyStartDate"] = earlyStart[0]
taskData["EarlyStartTime"] = earlyStart[1]
taskData.drop(columns = ["EarlyStart"], inplace = True)

dueDate = taskData["DueDate"].str.split("T", n = 1, expand = True)
taskData["DueDateDay"] = dueDate[0]
taskData["DueDateTime"] = dueDate[1]
taskData.drop(columns = ["DueDate"], inplace = True)

aptStart = taskData["AppointmentStart"].str.split("T", n = 1, expand = True)
taskData["appointmentStartDate"] = aptStart[0]
taskData["appointmentStartTime"] = aptStart[1]
taskData.drop(columns = ["AppointmentStart"], inplace = True)

aptFinish = taskData["AppointmentFinish"].str.split("T", n = 1, expand = True)
taskData["appointmentFinishDate"] = aptFinish[0]
taskData["appointmentFinishTime"] = aptFinish[1]
taskData.drop(columns = ["AppointmentFinish"], inplace = True)

#display(taskData)

In [5]:
#convert latitudes and longitudes into usable
taskData["Latitude"] = taskData["Latitude"].astype(str)
taskData["Longitude"] = taskData["Longitude"].astype(str)

taskData["lastSixLat"] = taskData["Latitude"].str[-6:]
taskData["trimmedLat"] = taskData["Latitude"].str[:-6]
taskData["Lat"] = taskData["trimmedLat"] + "." + taskData["lastSixLat"]
taskData.drop(columns = ["lastSixLat"], inplace = True)
taskData.drop(columns = ["trimmedLat"], inplace = True)
taskData.drop(columns = ["Latitude"], inplace = True)
taskData["Latitude"] = taskData["Lat"].astype(float)

taskData["lastSixLong"] = taskData["Longitude"].str[-6:]
taskData["trimmedLong"] = taskData["Longitude"].str[:-6]
taskData["Long"] = taskData["trimmedLong"] + "." + taskData["lastSixLong"]
taskData.drop(columns = ["lastSixLong"], inplace = True)
taskData.drop(columns = ["trimmedLong"], inplace = True)
taskData.drop(columns = ["Longitude"], inplace = True)
taskData["Longitude"] = taskData["Long"].astype(float)

display(taskData)

,SerialNo,ExternalRefID,CallID,CTLDispatchCallID,CTLAggregationID,Number,Priority,Area,Region,District,Postcode,TaskType,Duration,NumberOfRequiredEngineers,Street,City,State,CountryID,CTLDispatchReady,ES+duration,AS+duration,(AS+duration>=DD,(ES+duration>=DD,Duration (hh:mm:ss),DD-ES,ES>=DD,DD-AS,AS>=DD,AF>DD,Note,EarlyStartDate,EarlyStartTime,DueDateDay,DueDateTime,appointmentStartDate,appointmentStartTime,appointmentFinishDate,appointmentFinishTime,Lat,Latitude,Long,Longitude
0,1,213116,213116,MRNV1099624001F00,M/JUNETEST01/171231,1,4,NV,Las Vegas,East 2 NV,891156549.0,HSADSL2SANRIN,4800,1,NaN,NaN,NaN,UNITED STATES,False,10/30/2017 9:20,1:20:00 AM,OK,OK,1:20:00 AM,0/1/00 9:00,D,2017-10-30T17:00:00,D,D,NaN,2017-10-30,08:00:00,2017-10-30,17:00:00,NaN,NaN,NaN,NaN,36.196753,36.196753,-115.071860,-115.071860
1,2,253122,253122,MRNV4542934002A00,H/1445077137,1,4,NV,Las Vegas,North 2 NV,890311803.0,XXXXXXXXXXXIH,4260,1,NaN,NaN,NaN,UNITED STATES,True,10/27/2017 9:11,1:11:00 AM,OK,OK,1:11:00 AM,0/1/00 9:00,D,2017-10-27T17:00:00,D,D,NaN,2017-10-27,08:00:00,2017-10-27,17:00:00,NaN,NaN,NaN,NaN,36.272531,36.272531,-115.161580,-115.161580
2,3,253130,253130,MRNV4542934001A00,R/1445077137,1,4,NV,Las Vegas,North 2 NV,890311803.0,HSBNDA2FANRIN,14400,1,NaN,NaN,NaN,UNITED STATES,True,10/27/2017 12:00,4:00:00 AM,OK,OK,4:00:00 AM,0/1/00 9:00,D,2017-10-27T17:00:00,D,D,NaN,2017-10-27,08:00:00,2017-10-27,17:00:00,NaN,NaN,NaN,NaN,36.272531,36.272531,-115.161580,-115.161580
3,4,29319,29319,TINVC014723001PTD01,NaN,1,4,NV,Las Vegas,North 2 NV,89030.0,DSOCFEPREMDESX,6000,1,NaN,NaN,NaN,UNITED STATES,True,10/27/2017 10:40,1:40:00 AM,OK,OK,1:40:00 AM,0/1/00 9:00,D,2017-10-27T18:00:00,D,D,NaN,2017-10-27,09:00:00,2017-10-27,18:00:00,NaN,NaN,NaN,NaN,36.195789,36.195789,-115.118216,-115.118216
4,5,296995,296995,MRNV4306921001A00,R/1446647892,1,4,NV,Las Vegas,West West NV,891444443.0,XXXXXXXXXXXIH,4260,1,NaN,NaN,NaN,UNITED STATES,True,10/29/2017 8:11,2:11:00 PM,OK,OK,1:11:00 AM,0/1/00 12:00,D,1900-01-00T06:00:00,D,D,NaN,2017-10-29,07:00:00,2017-10-29,19:00:00,2017-10-29,13:00:00,2017-10-29,17:00:00,36.165665,36.165665,-115.331120,-115.331120
5,6,359391,359391,MRNV1244662001A00,R/1448416122,1,4,NV,Las Vegas,West West NV,891447009.0,HSBNDA2FANRIN,14400,1,NaN,NaN,NaN,UNITED STATES,True,10/28/2017 11:00,12:00:00 PM,OK,OK,4:00:00 AM,0/1/00 12:00,D,1900-01-00T11:00:00,D,D,NaN,2017-10-28,07:00:00,2017-10-28,19:00:00,2017-10-28,08:00:00,2017-10-28,17:00:00,36.186165,36.186165,-115.314710,-115.314710
6,7,394023,394023,MRNV4543918002A00,H/1449532307,1,4,NV,Las Vegas,South South NV,891230520.0,XXXXXXXXXXXIH,4260,1,NaN,NaN,NaN,UNITED STATES,True,10/30/2017 9:11,1:11:00 AM,OK,OK,1:11:00 AM,0/1/00 9:00,D,2017-10-30T17:00:00,D,D,NaN,2017-10-30,08:00:00,2017-10-30,17:00:00,NaN,NaN,NaN,NaN,36.050210,36.050210,-115.145440,-115.145440
7,8,394042,394042,MRNV4543918001A00,R/1449532307,1,4,NV,Las Vegas,South South NV,891230520.0,HSBNDA2FCHRIN,14280,1,NaN,NaN,NaN,UNITED STATES,True,10/30/2017 11:58,3:58:00 AM,OK,OK,3:58:00 AM,0/1/00 9:00,D,2017-10-30T17:00:00,D,D,NaN,2017-10-30,08:00:00,2017-10-30,17:00:00,NaN,NaN,NaN,NaN,36.050210,36.050210,-115.145440,-115.145440
8,9,394043,394043,MRNV4543918001A01,R/1449532307,1,4,NV,Las Vegas,South South NV,891230520.0,HSBNDA2FCHRIN,14280,1,NaN,NaN,NaN,UNITED STATES,True,10/30/2017 11:58,3:58:00 AM,OK,OK,3:58:00 AM,0/1/00 9:00,D,2017-10-30T17:00:00,D,D,NaN,2017-10-30,08:00:00,2017-10-30,17:00:00,NaN,NaN,NaN,NaN,36.050210,36.050210,-115.145440,-115.145440
9,10,421394,421394,TINVS035995001PTD01,NaN,1,4,NV,Las Vegas,West 8 NV,89146.0,DSOCFEPREMDESX,6360,1,NaN,NaN,NaN,UNITED STATES,True,10/30/2017 9:46,1:46:00 AM,OK,OK,1:46:00 AM,0/1/00 9:00,D,2017-10-30T17:00:00,D,D,NaN,2017-10-30,08:00:00,2017-10-30,17:00:00,NaN,NaN,NaN,NaN,36.159729,36.159729,-115.230590,-115.230590


In [6]:
distances = pd.DataFrame()
distances["Latitude"] = taskData.Latitude
distances["Longitude"] = taskData.Longitude
#distances['coords'] = zip(taskData.Latitude, taskData.Longitude)
distances.head()

,Latitude,Longitude
0,36.196753,-115.071860
1,36.272531,-115.161580
2,36.272531,-115.161580
3,36.195789,-115.118216
4,36.165665,-115.331120


In [13]:
#create an array of all distances
dfr = distances.copy()
dfr.Latitude = np.radians(distances.Latitude)
dfr.Longitude = np.radians(distances.Longitude)
hs = DistanceMetric.get_metric("haversine")
dist_array = (hs.pairwise(dfr)*6371) # Earth radius in km
display(dist_array)

array([[ 0.        , 11.65134605, 11.65134605, ...,  9.15447935,
        25.48237869, 25.48237869],
       [11.65134605,  0.        ,  0.        , ..., 12.29350007,
        26.92619503, 26.92619503],
       [11.65134605,  0.        ,  0.        , ..., 12.29350007,
        26.92619503, 26.92619503],
       ...,
       [ 9.15447935, 12.29350007, 12.29350007, ...,  0.        ,
        16.68470603, 16.68470603],
       [25.48237869, 26.92619503, 26.92619503, ..., 16.68470603,
         0.        ,  0.        ],
       [25.48237869, 26.92619503, 26.92619503, ..., 16.68470603,
         0.        ,  0.        ]])

In [14]:
#Calculate the time required to travel the distances and store to time array
#rate in kmph because distance is in km
rate = 56.327
#distance = rate * time so time = distance/rate
time_matrix = dist_array / 56.327
display(time_matrix)


array([[0.        , 0.20685188, 0.20685188, ..., 0.16252382, 0.45240078,
        0.45240078],
       [0.20685188, 0.        , 0.        , ..., 0.21825235, 0.47803354,
        0.47803354],
       [0.20685188, 0.        , 0.        , ..., 0.21825235, 0.47803354,
        0.47803354],
       ...,
       [0.16252382, 0.21825235, 0.21825235, ..., 0.        , 0.29621152,
        0.29621152],
       [0.45240078, 0.47803354, 0.47803354, ..., 0.29621152, 0.        ,
        0.        ],
       [0.45240078, 0.47803354, 0.47803354, ..., 0.29621152, 0.        ,
        0.        ]])

In [ ]:
#Compute the time windows
